In [ ]:
from flashrag.config import Config
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "5,"
config = Config(config_file_path="hotpotqa_baseline.yaml")

from flashrag.utils import get_dataset
dataset = get_dataset(config)

In [ ]:
from sayrag import generate_reflections
from vllm import LLM

reflection_path = 'test_reflections.jsonl'

if not os.path.isfile(reflection_path):

    llm = LLM(model="/shared/data3/yichi9/Mistral-7B-Instruct-v0.2-sayself", 
            max_model_len=2048,
            enable_lora=False,
            gpu_memory_utilization=0.7 )

    generate_reflections(llm, 
                        dataset, split='dev', key='question',
                        output='test_reflections.jsonl')

else:
    print("We found the reflection file!")

In [ ]:
from sayrag import SayRAGPipeline
all_split = get_dataset(config)
pipeline = SayRAGPipeline(config)

: 

In [ ]:
pipeline.run(
    dataset=dataset['dev'],
    reflection_path=reflection_path,
    ratio_augumented=1.0,
    min_naive_confidence=100,
    add_query_to_reflection=True,
)